In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.common.keys import Keys
from requests_html import HTMLSession
import pickle
import time
import random

In [2]:
with open("credentials.json", "rb") as file:
    data = pickle.loads(file.read())
    username, password = data["username"], data["password"]

In [3]:
url = "https://top-hashtags.com/instagram/"
trending_keyword = []

session = HTMLSession()
for i in range(22):
    if i != 0:
        url += str(i)+"01/"
    r = session.get(url)
    a_s = r.html.find("div.i-tag a", first=False)
    for a in a_s:
        trending_keyword.append(a.text)

print(trending_keyword)
    

['#love', '#instagood', '#fashion', '#photooftheday', '#beautiful', '#art', '#photography', '#happy', '#picoftheday', '#cute', '#follow', '#tbt', '#followme', '#nature', '#like4like', '#travel', '#instagram', '#style', '#repost', '#summer', '#instadaily', '#selfie', '#me', '#friends', '#fitness', '#girl', '#food', '#fun', '#beauty', '#instalike', '#smile', '#family', '#photo', '#life', '#likeforlike', '#music', '#ootd', '#follow4follow', '#makeup', '#amazing', '#igers', '#nofilter', '#dog', '#model', '#sunset', '#beach', '#instamood', '#foodporn', '#motivation', '#followforfollow', '#design', '#lifestyle', '#sky', '#l4l', '#f4f', '#일상', '#cat', '#handmade', '#hair', '#vscocam', '#bestoftheday', '#vsco', '#funny', '#dogsofinstagram', '#drawing', '#artist', '#gym', '#flowers', '#baby', '#wedding', '#girls', '#instapic', '#pretty', '#likeforlikes', '#photographer', '#instafood', '#party', '#inspiration', '#lol', '#cool', '#workout', '#likeforfollow', '#swag', '#fit', '#healthy', '#yummy',

In [4]:
len(trending_keyword)

2200

In [5]:
file = open("trending_keywords.txt", "w+")
for i in trending_keyword:
    file.write(i+"\n")
print("All Done [*_*]")

All Done [*_*]


In [6]:
driver = webdriver.FirefoxProfile()
driver.get("https://instagram.com/")


In [8]:
driver.find_element_by_css_selector("input[name='username']").clear()
driver.find_element_by_css_selector("input[name='password']").clear()
driver.find_element_by_css_selector("input[name='username']").send_keys(username)
driver.find_element_by_css_selector("input[name='password']").send_keys(password)
driver.find_element_by_css_selector("button.sqdOP.L3NKy.y3zKF[type='submit']").click()


In [9]:
# Saving the accoung login info btn handler...
driver.find_element_by_css_selector("button.sqdOP.L3NKy.y3zKF[type='button']").click()

In [10]:
# Handling the search button...
def search_hashtag(hashtag):
    """
    This function will search a give hashtag...
    """
    search_input = driver.find_element_by_css_selector("input.XTCLo.x3qfX[placeholder='Search']")
    search_input.clear()
    search_input.send_keys([hashtag])
    time.sleep(3)
    # We need to wait untill the AJAXify search menu appears then 
    # We'will click on the first one.
    # driver.find_element_by_css_selector("div.fuqBx a.yCE8d").click()
    # Let's use Web DriverWait instead of time.sleep()
    req_a = WebDriverWait(driver, 10).until(
        expected_conditions.presence_of_element_located((By.CSS_SELECTOR, "div.fuqBx a.yCE8d"))
    )
    req_a.click()
    

In [11]:
search_hashtag(trending_keyword[random.randint(0, len(trending_keyword)-1)])

In [12]:
base_comments = [
    "❤", "🥺", "✨", "🥰", "✨", "😊", "😍", "🏆", "🔥", "☀", "💕",
    "Coolest Post.",
    "Nice Post 🔥",
    "OMG Post.",
    "Nice", "Great Post 🏆"
]
LIKE_COUNTER = 0

def bot_log(msg):
    print("[BOT_LOG]: " + msg)

def get_random_comment(comments):
    return comments[random.randint(0, len(comments)-1)]

def like_hashtag_page(no_of_posts, comments, percentage_of_comments):
    """
    This function will try to find the clickable posts on the 
    current url, and click on the very first post.
    """
    global LIKE_COUNTER
    liked_color = "#ed4956"
    normal_color = "#262626"
    # driver.find_elements_by_css_selector("div.v1Nh3.kIKUG._bz0w")[0].click()
    post = WebDriverWait(driver, 10).until(
        expected_conditions.presence_of_element_located((By.CSS_SELECTOR, "div.v1Nh3.kIKUG._bz0w"))
    )
    post.click()
    # like_btn = driver.find_element_by_css_selector("span.fr66n button.wpO6b")
    if no_of_posts != "__all__":
        for i in range(no_of_posts):
            next_btn = driver.find_element_by_css_selector("a._65Bje.coreSpriteRightPaginationArrow")
            try:
                like_btn = WebDriverWait(driver, 10).until(
                    expected_conditions.presence_of_element_located((By.CSS_SELECTOR, "span.fr66n button.wpO6b"))
                )
            except:
                bot_log("THE POST SEEMS TO BE INVALID [WAITED FOR '10 sec']")
                next_btn.click()
                
            comment_input = driver.find_element_by_css_selector("form.X7cDz textarea.Ypffh")
            comment_input.click()
            comment_input.send_keys([get_random_comment(), Keys.ENTER])
            
            color = driver.find_element_by_css_selector("div.QBdPU  span svg._8-yf5").get_attribute("fill")
            if color == normal_color:
                like_btn.click()
                LIKE_COUNTER += 1
            else:
                print("[BOT_LOG]: FOUND POST ALREADY LIKED.")
            next_btn.click()
    else:
        while True:
            next_btn = driver.find_element_by_css_selector("a._65Bje.coreSpriteRightPaginationArrow")
            try:
                like_btn = WebDriverWait(driver, 10).until(
                    expected_conditions.presence_of_element_located((By.CSS_SELECTOR, "span.fr66n button.wpO6b"))
                )
            except :
                bot_log("THE POST SEEMS TO BE INVALID [WAITED FOR '10 sec']")
                next_btn.click()
            if random.randint(1, 100) > (100 - percentage_of_comments):
                driver.find_element_by_css_selector("span._15y0l button.wpO6b ").click()
                comment_input = driver.find_element_by_css_selector("textarea.Ypffh")
                comment_input.send_keys([get_random_comment(comments)])
                comment_input.send_keys(Keys.ENTER)
            color = driver.find_element_by_css_selector("div.QBdPU  span svg._8-yf5").get_attribute("fill")
            if color == normal_color:
                like_btn.click()
                LIKE_COUNTER += 1
            else:
                print("[BOT_LOG]: FOUND POST ALREADY LIKED.")
            next_btn.click()
    return LIKE_COUNTER
            

    


In [14]:
# like_hashtag_page("__all__")

In [15]:
def generate_random_hashtag():
    return trending_keyword[random.randint(0, len(trending_keyword)-1)]

In [16]:
# search_hashtag("#100daysofcode")
# # time.sleep(5)
# like_hashtag_page("__all__")

In [17]:


comments = [
    "❤", "🥺", "✨", "🥰", "✨", "😊", "😍", "🏆", "🔥", "☀", "💕",
    "Coolest Post.",
    "Nice Post 🔥",
    "OMG Post.",
    "Nice", "Great Post 🏆"
]

def scrap_comments():
    """
    URL: https://www.trendingus.com/comments-for-girls/
    This function scraps some of the best comments from the internet.
    """
    r = session.get("https://www.trendingus.com/comments-for-girls/")
    li_s = r.html.find("ol li", first=False)
    for li in li_s:
        comments.append(li.text + ", Just Kidding 😊")
scrap_comments()
print(comments, "\nLength: ", len(comments))        

['❤', '🥺', '✨', '🥰', '✨', '😊', '😍', '🏆', '🔥', '☀', '💕', 'Coolest Post.', 'Nice Post 🔥', 'OMG Post.', 'Nice', 'Great Post 🏆', 'I think this is the best I’ve seen till now, Just Kidding 😊', 'Just when I couldn’t love you more. You posted this pic and my jaw dropped to the floor., Just Kidding 😊', 'You are a symbol of beauty, Just Kidding 😊', 'Surely you are the most beautiful woman alive, the most that I’ve seen, Just Kidding 😊', 'You mean the world to me., Just Kidding 😊', 'The word pretty is worthless without you, Just Kidding 😊', 'You are the definition of “Beauty”, Just Kidding 😊', 'Flawless outstanding eccentric lovely beauty, Just Kidding 😊', 'The stars, the moon, and the sun are minor to me since you sparkle brighter than all of them, Just Kidding 😊', 'You are graceful, Just Kidding 😊', 'Surely you would have been arrested if looking immensely beautiful was a crime., Just Kidding 😊', 'You’re an incredibly stunning, a really pretty, beautiful woman, Just Kidding 😊', 'I guess this i

In [ ]:

# like_hashtag_page(20, base_comments, 10)

In [19]:
def generate_random_comment_emoji_list(emoji_list, length_of_comment, no_of_comments):
    """
    emoji_list: a list of emoji from which we generate the comment
    length_of_comment: self explanatory...
    no_of_comments: no of comments you wanna generate
    """
    comment_list = []
    for j in range(no_of_comments):
        comment = ""
        for i in range(length_of_comment):    
            comment += emoji_list[random.randint(0, len(emoji_list)-1)]
        comment_list.append(comment)
    return comment_list

print(generate_random_comment_emoji_list(["❤", "🥺", "✨", "🥰", "✨", "😊", "😍", "🏆", "🔥", "☀", "💕"], 5, 100))        
        

['❤😍✨😍☀', '🥰❤😍❤❤', '✨🥺💕😍☀', '❤🥰✨❤✨', '✨🥺🥰😍😊', '✨✨😍☀🥺', '🥰🥺😍🥺🥺', '🥰🏆💕🏆😊', '🥺🥺🥺🥰💕', '✨🏆✨🏆😊', '☀✨✨🏆😊', '✨🥺🥰☀❤', '✨✨💕🥺☀', '😊🔥😊🥰✨', '💕😊☀❤💕', '🥰🏆❤✨🥰', '☀❤😍🔥🔥', '🔥🏆✨😍🏆', '🥺🥺🥰🥰😊', '😍🥰💕✨😍', '❤🥺❤🥺🔥', '✨💕🥺🥺❤', '🥰😍✨✨☀', '🔥☀💕✨🥺', '❤✨🔥🔥☀', '🥺🥰🥺✨🏆', '🔥🥺😊✨☀', '🔥☀☀✨🥺', '😊☀😊✨💕', '🥺❤🥺🥺✨', '☀😊😍🥰🏆', '😊😊😍💕💕', '😊✨😊🏆✨', '💕✨☀✨🥺', '✨✨☀✨✨', '🔥✨✨✨😊', '🥰🥰✨✨🔥', '🏆😊🥰😍🥺', '✨🥺☀❤🥺', '✨❤✨💕❤', '😊😊🥰✨🔥', '✨😊😍🔥🏆', '😊🏆😊💕☀', '✨🔥🥰☀🥺', '❤🥰😍🏆😊', '☀🥰☀✨😍', '☀🥰🔥✨🥺', '✨🥺🔥☀✨', '😍☀❤❤😍', '💕✨💕🥺🏆', '❤✨✨🥺🏆', '✨🥺☀☀😊', '✨💕☀☀🏆', '🥺✨🥺✨❤', '🏆🥺😍🥺🥰', '💕✨❤❤🏆', '🥺🥺✨😊✨', '😍🥰💕❤😍', '❤🏆😊💕🥰', '❤✨🔥💕🔥', '☀✨🔥✨🥰', '✨😊✨😍💕', '✨😊❤😍🏆', '✨🔥❤❤✨', '😊🥺💕😍🔥', '✨🏆❤❤💕', '🥰💕✨🔥💕', '☀☀🥺🏆☀', '😊☀🥰😊😊', '😍💕🥺🥰😊', '✨✨🏆✨😊', '✨🥺☀🔥✨', '🥰✨🔥🥺☀', '😍🥺🔥🥰☀', '🏆🏆😍❤😍', '😍🥰☀🥺😊', '✨❤🏆🏆🥺', '✨❤🥺✨🥺', '🥰🔥✨🏆🏆', '🏆✨❤😊🥺', '🥰😍🥰🏆😊', '💕🏆🔥❤🔥', '☀🥰✨☀✨', '😊😍🥰🔥✨', '✨☀🥰✨✨', '🏆😊❤✨🏆', '❤✨☀🥰😊', '🥰✨🥰🔥🥰', '🥺🥺❤✨❤', '😍✨☀☀🥺', '❤😍🏆☀☀', '💕🔥✨💕🥰', '✨✨☀🔥🏆', '🥰😊😊☀🥰', '😍😍✨🔥🥺', '🏆😍☀☀❤', '✨🔥❤✨😊', '😊🏆✨🏆🥺', '💕❤💕🥺🥰', '🏆🏆🔥❤☀']


In [22]:
search_hashtag("#python3")
time.sleep(5)
count = like_hashtag_page(
    no_of_posts="__all__", 
    comments=generate_random_comment_emoji_list(
        emoji_list=["❤", "🥺", "✨", "🥰", "✨", "😊", "😍", "🏆", "🔥", "☀", "💕"], 
        length_of_comment=5, 
        no_of_comments=1000
    ), 
    percentage_of_comments=5
)
print(count)

[BOT_LOG]: FOUND POST ALREADY LIKED.
[BOT_LOG]: FOUND POST ALREADY LIKED.
[BOT_LOG]: FOUND POST ALREADY LIKED.
[BOT_LOG]: FOUND POST ALREADY LIKED.
[BOT_LOG]: FOUND POST ALREADY LIKED.
[BOT_LOG]: FOUND POST ALREADY LIKED.
[BOT_LOG]: FOUND POST ALREADY LIKED.
[BOT_LOG]: FOUND POST ALREADY LIKED.
[BOT_LOG]: FOUND POST ALREADY LIKED.
[BOT_LOG]: FOUND POST ALREADY LIKED.
[BOT_LOG]: THE POST SEEMS TO BE INVALID [WAITED FOR '10 sec']


NoSuchElementException: Message: Unable to locate element: div.QBdPU  span svg._8-yf5
